In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from tensorflow import keras 
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.applications.mobilenet import preprocess_input
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter
from tensorflow.keras.applications.vgg19 import VGG19
print("Num of GPUs available: ", len(tf.test.gpu_device_name()))
import os
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from PIL import Image

2.9.1
Num of GPUs available:  13


2022-05-26 11:20:53.509843: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-26 11:20:53.546393: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-26 11:20:53.549952: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-26 11:20:53.550247: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [3]:
datagen = ImageDataGenerator()

Path = "/home/atik/Documents/Summer_1/data/"
train_generator = datagen.flow_from_directory(
    directory=Path+"/train/",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=64,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

val_generator = datagen.flow_from_directory(
    directory=Path+"/val/",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=64,
    class_mode="categorical",
    shuffle=True,
    seed=42
)
counter = Counter(train_generator.classes)
img, _ = train_generator.next()
img = np.array(img[0])
shape = np.shape(img)

Found 25600 images belonging to 256 classes.
Found 5120 images belonging to 256 classes.


In [4]:
# def fire_module(x,s1,e1,e3):
#     s1x = tf.keras.layers.Conv2D(s1,kernel_size = 1, padding = 'same')(x)
#     s1x = tf.keras.layers.ReLU()(s1x)
#     e1x = tf.keras.layers.Conv2D(e1,kernel_size = 1, padding = 'same')(s1x)
#     e3x = tf.keras.layers.Conv2D(e3,kernel_size = 3, padding = 'same')(s1x)
#     x = tf.keras.layers.concatenate([e1x,e3x])
#     x = tf.keras.layers.ReLU()(x)
#     return x

# def SqueezeNet(input_shape, nclasses):
#     input = tf.keras.layers.Input(input_shape)
#     x = tf.keras.layers.Conv2D(96,kernel_size=(7,7),strides=(2,2),padding='same')(input)
#     x = tf.keras.layers.MaxPool2D(pool_size=(3,3), strides = (2,2))(x)
#     x = fire_module(x, s1 = 16, e1 = 64, e3 = 64)
#     x = fire_module(x, s1 = 16, e1 = 64, e3 = 64)
#     x = fire_module(x, s1 = 32, e1 = 128, e3 = 128)
#     x = tf.keras.layers.MaxPool2D(pool_size=(3,3), strides = (2,2))(x)
#     x = fire_module(x, s1 = 32, e1 = 128, e3 = 128)
#     x = fire_module(x, s1 = 48, e1 = 192, e3 = 192)
#     x = fire_module(x, s1 = 48, e1 = 192, e3 = 192)
#     x = fire_module(x, s1 = 64, e1 = 256, e3 = 256)
#     x = tf.keras.layers.MaxPool2D(pool_size=(3,3), strides = (2,2))(x)
#     x = fire_module(x, s1 = 64, e1 = 256, e3 = 256)
#     x = tf.keras.layers.Dropout(0.5)(x)
#     x = tf.keras.layers.Conv2D(nclasses,kernel_size = 1)(x)
#     output = tf.keras.layers.AvgPool2D(pool_size=(13,13))(x)
#     model = tf.keras.Model(input, output)
#     return model

In [4]:
model = VGG19(weights=None, classes=len(counter))
model.count_params()

2022-05-26 11:21:03.950184: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-26 11:21:03.950574: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-26 11:21:03.950829: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-26 11:21:03.951211: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-26 11:21:03.951470: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from S

140619072

In [5]:
model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=1e-2, momentum=0.9, nesterov=True), 
    loss='categorical_crossentropy', metrics=['accuracy', 'top_k_categorical_accuracy']
)

In [ ]:
model.fit(
    train_generator, 
    steps_per_epoch=400, epochs=30, verbose=1,
    callbacks=[
        ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, patience=2, epsilon=0.007),
        EarlyStopping(monitor='val_accuracy', patience=4, min_delta=0.01)
    ],
    validation_data=val_generator, validation_steps=80, workers=4
)

Epoch 1/30


2022-05-26 11:21:11.527187: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2022-05-26 11:21:12.010233: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-05-26 11:21:12.010631: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-05-26 11:21:12.010647: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2022-05-26 11:21:12.011016: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-05-26 11:21:12.011056: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2022-05-26 11:21:13.418094: I tensorflow/stream_executor/cuda/c

266/400 [==================>...........] - ETA: 34s - loss: 5.5458 - accuracy: 0.0041 - top_k_categorical_accuracy: 0.0189

In [ ]:
plt.plot(model.history.history['loss'], label='train');
plt.plot(model.history.history['val_loss'], label='val');
plt.legend();
plt.xlabel('epoch');
plt.ylabel('logloss');

In [ ]:
plt.plot(model.history.history['accuracy'], label='train');
plt.plot(model.history.history['val_accuracy'], label='val');
plt.legend();
plt.xlabel('epoch');
plt.ylabel('accuracy');

In [ ]:
plt.plot(model.history.history['top_k_categorical_accuracy'], label='train');
plt.plot(model.history.history['val_top_k_categorical_accuracy'], label='val');
plt.legend();
plt.xlabel('epoch');
plt.ylabel('top5_accuracy');

In [ ]:
model.evaluate_generator(val_generator, 80)